# Comparisons of differences in brain regions

This notebook performs omnibus embedding of volumetric and fractional antisotropy difference matrices for mice. For each mouse, we have a matrix consisting of the average volume and fractional antisotropy in a given region. These data matrices are first normalized across mice. Using the normalized measurements, compute the pairwise differences between all pairs of regions to form distance matrices, whose entries indicate the difference in the indicated measurement type (volume; mean FA) between a given pair of regions. These distance matrices are then embedded using the omnibus embedding into 18 embedding dimensions. For each mouse, this gives us an $N \times d$ estimated latent position matrix, whose entries correspond to the estimated $d$-dimensional latent position for each region of interest in the mouse data.

In [1]:
import pandas as pd
from graspologic.embed import OmnibusEmbed
from scipy.spatial.distance import pdist, squareform

In [2]:
volumes = pd.read_csv("../data/processed/volumes.csv")
fa = pd.read_csv("../data/processed/fa.csv")
meta = pd.read_csv("../data/processed/meta.csv")

vols = []
for idx, row in meta.iterrows():
    ID = row["ID"]
    vols.append(volumes[[ID]].values.ravel())

fas = []
for idx, row in meta.iterrows():
    ID = row["ID"]
    fas.append(fa[[ID]].values.ravel())

vols = np.vstack(vols)
vols /= vols.sum(axis=1, keepdims=True)

fas = np.vstack(fas)
fas /= fas.sum(axis=1, keepdims=True)

vol_dists = [squareform(pdist(v.reshape(-1, 1))) for v in vols]
fa_dists = [squareform(pdist(v.reshape(-1, 1))) for v in fas]

In [3]:
omni = OmnibusEmbed(n_components=18)
vol_embed = omni.fit_transform(vol_dists)

In [70]:
omni = OmnibusEmbed(n_components=18)
fa_embed = omni.fit_transform(fa_dists)

In [44]:
np.save("../data/embeddings/vol_embed", vol_embed)

In [71]:
np.save("../data/embeddings/fa_embed", fa_embed)